# Project Overview

This project focuses on analyzing sales data using SQL to demonstrate
both fundamental and intermediate querying skills. The analysis is
performed on a relational database containing customers, products,
and orders.

The project covers data exploration, aggregations, multi-table joins,
revenue analysis, customer and order segmentation, and time-based
trend analysis. The goal is to answer real-world business questions
using clean and efficient SQL queries.


# Tools & Technologies

- SQL (SQLite)
- Python (used to execute SQL queries)
- Pandas
- Jupyter Notebook

In [2]:
import pandas as pd
import sqlite3

# Database Setup & Connection

In [3]:
conn= sqlite3.connect("sales.db")
cursor= conn.cursor()

#  Data Model & Create Tables

### 1. Customers Table

In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
       customer_id INTEGER PRIMARY KEY,
       name TEXT,
       city TEXT
)
""")

### 2. Products Table

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
       product_id INTEGER PRIMARY KEY,
       product_name TEXT,
       price INTEGER
)
""")

### 3. Orders Table

In [6]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
       order_id INTEGER PRIMARY KEY,
       customer_id INTEGER,
       product_id INTEGER,
       quantity INTEGER,
       order_date TEXT
)
""")

# Data Reset & Reproducibility

In [47]:
# Reset data to avoid duplicates when re-running the notebook
cursor.execute("DELETE FROM orders")
cursor.execute("DELETE FROM customers")
cursor.execute("DELETE FROM products")
conn.commit()

# Data Insertion

### 1. Insert Customers

In [48]:
cursor.executemany("""
INSERT INTO customers (name, city)
VALUES (?, ?)
""", [
    ("Amit", "Delhi"),
    ("Rahul", "Mumbai"),
    ("Sneha", "Bangalore"),
    ("Priya", "Delhi")
])

### 2. Insert Products

In [49]:
cursor.executemany("""
INSERT INTO products (product_name, price)
VALUES (?, ?)
""", [
    ("Laptop", 50000),
    ("Phone", 20000),
    ("Headphones", 3000)
])

### 3. Insert Orders

In [50]:
cursor.executemany("""
INSERT INTO orders (customer_id, product_id, quantity, order_date)
VALUES (?, ?, ?, ?)
""", [
    (1, 1, 1, "2025-12-10"),
    (2, 2, 2, "2025-12-12"),
    (3, 3, 3, "2025-12-15"),
    (1, 2, 1, "2025-12-20"),
    (4, 1, 1, "2025-12-22")
])


In [51]:
conn.commit()

# Exploratory SQL Queries

In [52]:
pd.read_sql("SELECT * FROM customers", conn)

,customer_id,name,city
0,1,Amit,Delhi
1,2,Rahul,Mumbai
2,3,Sneha,Bangalore
3,4,Priya,Delhi


In [53]:
pd.read_sql("""
SELECT * FROM customers WHERE city = 'Delhi'
""", conn)

,customer_id,name,city
0,1,Amit,Delhi
1,4,Priya,Delhi


In [54]:
pd.read_sql("""
SELECT COUNT(*) AS total_customers
FROM customers
""", conn)

,total_customers
0,4


In [55]:
pd.read_sql("""
SELECT customer_id, SUM(quantity) AS total_items
FROM orders
GROUP BY customer_id
""", conn)

,customer_id,total_items
0,1,2
1,2,2
2,3,3
3,4,1


In [56]:
pd.read_sql("""
SELECT product_id,SUM(quantity) as total_orders
FROM ORDERS
GROUP BY product_id
""",conn)

,product_id,total_orders
0,1,2
1,2,3
2,3,3


In [57]:
pd.read_sql("""
SELECT COUNT(*) AS total_orders
FROM orders
""", conn)

,total_orders
0,5


In [58]:
pd.read_sql("""
SELECT *
FROM products
WHERE price > 10000
""", conn)

,product_id,product_name,price
0,1,Laptop,50000
1,2,Phone,20000


# Multi-Table JOIN Analysis

In [59]:
# Show all orders with customer name and product name.
pd.read_sql("""
SELECT o.order_id, c.name AS customer_name, p.product_name, o.quantity
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN products p ON o.product_id = p.product_id
""", conn)

,order_id,customer_name,product_name,quantity
0,1,Amit,Laptop,1
1,2,Rahul,Phone,2
2,3,Sneha,Headphones,3
3,4,Amit,Phone,1
4,5,Priya,Laptop,1


In [60]:
# Show all customers and any orders they made (include customers with no orders)
# Great for finding inactive customers
pd.read_sql("""
SELECT c.name AS customer_name, o.order_id, o.quantity
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
""", conn)

,customer_name,order_id,quantity
0,Amit,1,1
1,Amit,4,1
2,Rahul,2,2
3,Sneha,3,3
4,Priya,5,1


# Business Performance Analysis

### 1. Total Quantity Sold by Product

In [61]:
pd.read_sql("""
SELECT c.city,SUM(o.quantity) AS total_quantity_sold
FROM orders o
INNER JOIN customers c ON o.customer_id= c.customer_id
GROUP BY c.city
ORDER BY total_quantity_sold DESC
""", conn)

,city,total_quantity_sold
0,Delhi,3
1,Bangalore,3
2,Mumbai,2


### 2. Total Revenue Sold by City

In [62]:
pd.read_sql("""
SELECT
    c.city,
    SUM(p.price * o.quantity) AS total_revenue
FROM orders o
INNER JOIN customers c
    ON o.customer_id = c.customer_id
INNER JOIN products p
    ON o.product_id = p.product_id
GROUP BY c.city
ORDER BY total_revenue DESC
""", conn)

,city,total_revenue
0,Delhi,120000
1,Mumbai,40000
2,Bangalore,9000


### 3. Average Order Value per cutomer

In [63]:
pd.read_sql("""
SELECT
    c.name,
    AVG(p.price * o.quantity) AS avg_order_value
FROM orders o
INNER JOIN customers c
    ON o.customer_id = c.customer_id
INNER JOIN products p
    ON o.product_id = p.product_id
GROUP BY c.name
ORDER BY avg_order_value DESC
""", conn)

,name,avg_order_value
0,Priya,50000.0
1,Rahul,40000.0
2,Amit,35000.0
3,Sneha,9000.0


# Order Value & Customer Spend Analysis

In [67]:
pd.read_sql("""
SELECT
    AVG(p.price * o.quantity) AS avg_order_value,
    MIN(p.price * o.quantity) AS min_order_value,
    MAX(p.price * o.quantity) AS max_order_value
FROM orders o
INNER JOIN products p
    ON o.product_id = p.product_id
""", conn)

,avg_order_value,min_order_value,max_order_value
0,33800.0,9000,50000


# Order & Customer Segmentation Using CASE WHEN

In [66]:
pd.read_sql("""
SELECT
    o.order_id,
    c.name AS customer_name,
    p.product_name,
    p.price * o.quantity AS order_value,
    CASE
        WHEN p.price * o.quantity >= 30000 THEN 'High Value'
        ELSE 'Low Value'
    END AS order_category
FROM orders o
INNER JOIN customers c
    ON o.customer_id = c.customer_id
INNER JOIN products p
    ON o.product_id = p.product_id
ORDER BY order_value DESC
""", conn)

,order_id,customer_name,product_name,order_value,order_category
0,1,Amit,Laptop,50000,High Value
1,5,Priya,Laptop,50000,High Value
2,2,Rahul,Phone,40000,High Value
3,4,Amit,Phone,20000,Low Value
4,3,Sneha,Headphones,9000,Low Value


In [64]:
pd.read_sql("""
SELECT
    c.name,
    SUM(p.price * o.quantity) AS total_spent,
    CASE
        WHEN SUM(p.price * o.quantity) >= 50000 THEN 'High Value Customer'
        ELSE 'Regular Customer'
    END AS customer_type
FROM orders o
INNER JOIN customers c
    ON o.customer_id = c.customer_id
INNER JOIN products p
    ON o.product_id = p.product_id
GROUP BY c.name
ORDER BY total_spent DESC
""", conn)

,name,total_spent,customer_type
0,Amit,70000,High Value Customer
1,Priya,50000,High Value Customer
2,Rahul,40000,Regular Customer
3,Sneha,9000,Regular Customer


# Time-Based Revenue Analysis

In [65]:
pd.read_sql("""
SELECT
    strftime('%Y-%m', o.order_date) AS month,
    c.city,
    SUM(p.price * o.quantity) AS monthly_revenue
FROM orders o
INNER JOIN customers c
    ON o.customer_id = c.customer_id
INNER JOIN products p
    ON o.product_id = p.product_id
GROUP BY month, c.city
ORDER BY month, monthly_revenue DESC
""", conn)

,month,city,monthly_revenue
0,2025-12,Delhi,120000
1,2025-12,Mumbai,40000
2,2025-12,Bangalore,9000


# Key Business Insights

- Cities with higher laptop purchases generated significantly more revenue,
  highlighting the strong impact of high-priced products on overall sales.

- Although some products had higher order quantities, revenue contribution
  was driven primarily by product price rather than volume alone.

- High-value orders (≥ 30,000) contributed a disproportionate share of total
  revenue, indicating the importance of premium transactions.

- Customer spending patterns varied noticeably, with a small number of
  customers accounting for a larger share of total revenue.

- Monthly revenue analysis showed clear time-based trends, demonstrating the
  importance of monitoring performance over time rather than relying on
  cumulative totals.


# Conclusion

This project demonstrates the application of SQL to analyze relational sales
data and answer practical business questions. Through the use of joins,
aggregations, conditional logic, and time-based analysis, meaningful insights
were derived from raw transactional data. The same approach can be extended to
larger datasets and integrated with visualization or BI tools.
